In [ ]:
%pip show webdriver-manager
%pip show selenium

In [1]:
import csv
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome() # opens up the web page
driver.get("https://cases.shc.gov.pk/") # sindh high court website